# Evaluation of MSSW on SEA

## Imports

In [1]:
import numpy as np
import pandas as pd
import sklearn

## SEA dataset locations

In [2]:
abrupt_sea_path = '../Datasets_concept_drift/synthetic_data/abrupt_drift/sea_1_abrupt_drift_0_noise_balanced.arff'
gradual_sea_paths = [
    '../Datasets_concept_drift/synthetic_data/gradual_drift/sea_1_gradual_drift_0_noise_balanced_05.arff',
    '../Datasets_concept_drift/synthetic_data/gradual_drift/sea_1_gradual_drift_0_noise_balanced_1.arff',
    '../Datasets_concept_drift/synthetic_data/gradual_drift/sea_1_gradual_drift_0_noise_balanced_5.arff',
    '../Datasets_concept_drift/synthetic_data/gradual_drift/sea_1_gradual_drift_0_noise_balanced_10.arff',
    '../Datasets_concept_drift/synthetic_data/gradual_drift/sea_1_gradual_drift_0_noise_balanced_20.arff'
]
all_sea_data_paths = [abrupt_sea_path] + gradual_sea_paths

## Accept and preprocess SEA datasets

In [3]:
from eval_helpers import accepting
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


sea_reference_batches = {}
sea_testing_batches = {}
for sea_path in all_sea_data_paths:
    df_x, df_y = accepting.get_clean_df(sea_path)
    df_y = pd.DataFrame(LabelEncoder().fit_transform(df_y))

    df_x_ref, df_x_test, df_y_ref, df_y_test = sklearn.model_selection.train_test_split(
        df_x, df_y, test_size=0.7, shuffle=False)
    
    reference_data = df_x_ref.to_numpy()
    testing_data = df_x_test.to_numpy()
    num_ref_batches = 3
    num_test_batches = 7
    ref_batches = np.array_split(reference_data, num_ref_batches)
    test_batches = np.array_split(testing_data, num_test_batches)
    
    sea_reference_batches[sea_path] = ref_batches
    sea_testing_batches[sea_path] = test_batches

df         attrib1   attrib2   attrib3      class
0      7.308782  4.100808  2.077148  b'groupB'
1      5.833539  0.422983  7.616747  b'groupA'
2      1.397627  6.949480  8.052278  b'groupB'
3      2.750299  0.753878  6.105915  b'groupA'
4      2.049135  6.233638  1.847071  b'groupB'
...         ...       ...       ...        ...
99995  4.636430  6.639370  0.066740  b'groupB'
99996  4.251993  3.351235  5.652197  b'groupA'
99997  4.131405  6.371722  3.125554  b'groupB'
99998  1.404214  4.392506  9.298558  b'groupA'
99999  7.231749  8.770465  3.925490  b'groupB'

[100000 rows x 4 columns]


C:\Users\jpohl\PycharmProjects\clustering-drift-detection\mssw\eval_helpers\accepting.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].str.decode('utf-8')
c:\users\jpohl\pycharmprojects\clustering-drift-detection\venv\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


df         attrib1   attrib2   attrib3      class
0      7.308782  4.100808  2.077148  b'groupB'
1      5.833539  0.422983  7.616747  b'groupA'
2      1.397627  6.949480  8.052278  b'groupB'
3      2.750299  0.753878  6.105915  b'groupA'
4      2.049135  6.233638  1.847071  b'groupB'
...         ...       ...       ...        ...
99995  2.074508  1.775662  1.318589  b'groupA'
99996  4.636430  6.639370  0.066740  b'groupB'
99997  4.251993  3.351235  5.652197  b'groupA'
99998  4.131405  6.371722  3.125554  b'groupB'
99999  1.404214  4.392506  9.298558  b'groupA'

[100000 rows x 4 columns]


C:\Users\jpohl\PycharmProjects\clustering-drift-detection\mssw\eval_helpers\accepting.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].str.decode('utf-8')
c:\users\jpohl\pycharmprojects\clustering-drift-detection\venv\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


df         attrib1   attrib2   attrib3      class
0      7.308782  4.100808  2.077148  b'groupB'
1      5.833539  0.422983  7.616747  b'groupA'
2      1.397627  6.949480  8.052278  b'groupB'
3      2.750299  0.753878  6.105915  b'groupA'
4      2.049135  6.233638  1.847071  b'groupB'
...         ...       ...       ...        ...
99995  2.261206  1.404770  3.977088  b'groupA'
99996  0.795885  8.915077  6.892585  b'groupB'
99997  0.353597  1.289198  0.001943  b'groupA'
99998  6.540503  5.698432  7.743243  b'groupB'
99999  2.074508  1.775662  1.318589  b'groupA'

[100000 rows x 4 columns]


...

C:\Users\jpohl\PycharmProjects\clustering-drift-detection\mssw\eval_helpers\accepting.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].str.decode('utf-8')
c:\users\jpohl\pycharmprojects\clustering-drift-detection\venv\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


df         attrib1   attrib2   attrib3      class
0      7.308782  4.100808  2.077148  b'groupB'
1      5.833539  0.422983  7.616747  b'groupA'
2      1.397627  6.949480  8.052278  b'groupB'
3      2.750299  0.753878  6.105915  b'groupA'
4      2.049135  6.233638  1.847071  b'groupB'
...         ...       ...       ...        ...
99995  8.630346  3.981509  1.040496  b'groupB'
99996  0.656783  5.144250  9.648631  b'groupA'
99997  8.299312  7.466245  5.592216  b'groupB'
99998  2.728767  5.140916  0.727831  b'groupA'
99999  8.852912  3.855575  6.128628  b'groupB'

[100000 rows x 4 columns]


C:\Users\jpohl\PycharmProjects\clustering-drift-detection\mssw\eval_helpers\accepting.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].str.decode('utf-8')
c:\users\jpohl\pycharmprojects\clustering-drift-detection\venv\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


df         attrib1   attrib2   attrib3      class
0      7.308782  4.100808  2.077148  b'groupB'
1      5.833539  0.422983  7.616747  b'groupA'
2      1.397627  6.949480  8.052278  b'groupB'
3      2.750299  0.753878  6.105915  b'groupA'
4      2.049135  6.233638  1.847071  b'groupB'
...         ...       ...       ...        ...
99995  8.479838  4.037801  7.474048  b'groupB'
99996  0.587347  2.725972  8.395393  b'groupA'
99997  1.297264  9.227339  6.843533  b'groupB'
99998  2.009023  5.305785  1.423271  b'groupA'
99999  9.680480  4.642564  3.628668  b'groupB'

[100000 rows x 4 columns]


C:\Users\jpohl\PycharmProjects\clustering-drift-detection\mssw\eval_helpers\accepting.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].str.decode('utf-8')
c:\users\jpohl\pycharmprojects\clustering-drift-detection\venv\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


## Find the best tol and max_iter in SEA (the drift type is irrelevant)

In [9]:
from core import mssw_preprocessing
from eval_helpers import kmeans_verbose_helpers


weighted_joined_reference_data, _, _ = mssw_preprocessing.mssw_preprocess(
    sea_reference_batches[abrupt_sea_path], sea_testing_batches[abrupt_sea_path])

filename = 'sea_new_output.txt'
kmeans_verbose_helpers.write_verbose_kmeans_to_file(filename, weighted_joined_reference_data,
                             n_clusters=2, n_init=100, max_iter=500, tol=0, random_state=1053)
output_dicts = kmeans_verbose_helpers.convert_kmeans_output_file_to_dicts(filename, n_init=100)
kmeans_verbose_helpers.print_stats_from_kmeans_output_dicts(output_dicts)

random state: 1053
total number of results: 100
maximum number of iterations: 85
minimum initial inertia: 609.7320029968274
maximum initial inertia: 1517.7243528803774
number of unique final inertia values: 14
minimum final inertia: 574.9767008337368
maximum final inertia: 575.2536701263556
total number of convergences: 100
number of strict convergences: 100
number of tol-based convergences: 0


## Use them for the analysis

In [10]:
from core import mssw_eval


sea_stats = {}
for sea_path in all_sea_data_paths:
    runs_results_bool, final_fpr_mean, fpr_std_err, final_latency_mean, latency_std_err = \
        mssw_eval.all_drifting_batches_randomness_robust(
        sea_reference_batches[sea_path],
        sea_testing_batches[sea_path],
        n_clusters=2,
        n_init=100,
        max_iter=850,
        tol=0,
        true_drift_idx=2,
        min_runs=2
    )
    sea_stats[sea_path] = {
        'runs_results_bool': runs_results_bool,
        'final_fpr_mean': final_fpr_mean,
        'fpr_std_err': fpr_std_err,
        'final_latency_mean': final_latency_mean,
        'latency_std_err': latency_std_err
    }

print('SEA STATS')
print(sea_stats)

min_runs 2
centroid distance sums [[1326.54998869  928.28742733]]
num points in clusters [[6221. 3779.]]
centroid distance sums [[1300.43570709  936.1917875 ]]
num points in clusters [[6135. 3865.]]
centroid distance sums [[1332.76755999  912.49976589]]
num points in clusters [[6241. 3759.]]
centroid distance sums [[1305.35614315  939.83355625]]
num points in clusters [[6171. 3829.]]
centroid distance sums [[1330.59494475  916.90438569]]
num points in clusters [[6266. 3734.]]
centroid distance sums [[1223.69720501 1068.20475581]]
num points in clusters [[5620. 4380.]]
centroid distance sums [[1146.59442582 1211.62094112]]
num points in clusters [[5070. 4930.]]
centroid distance sums [[1135.22052899 1204.89729334]]
num points in clusters [[5045. 4955.]]
centroid distance sums [[1138.96834908 1206.66183404]]
num points in clusters [[5080. 4920.]]
centroid distance sums [[1154.56913499 1200.64888159]]
num points in clusters [[5117. 4883.]]
centroid distance sums [[1326.98532168  927.86452

In [12]:
from eval_helpers import helpers


final_result_dict = {
    'type_of_data': [], 'dataset': [], 'drift': [], 'width': [], 'encoding': [],
    'n_init': [], 'max_iter': [], 'tol': [],
    'FPR_mean': [], 'latency_mean': []
}

for data_path, stats_dict in sea_stats.items():
#     data_filename = data_path.split('/')[-1]
#     type_of_data = data_path.split('/')[2].split('_')[0]  # synthetic or real-world
#     dataset_name = data_filename.split('_')[0]  # sea, agraw1, agraw2
#     drift_type = data_path.split('/')[3].split('_')[0]
#     drift_width = '0' if drift_type == 'abrupt' else data_filename.split('_')[-1].split('.')[0]
#     drift_width = 0.5 if drift_width == '05' else float(drift_width)
    synthetic_filename_info = helpers.synthetic_data_information(data_path)
    encoding = 'exclude'
    fpr_mean = float(stats_dict['final_fpr_mean'])
    latency_mean = float(stats_dict['final_latency_mean'])
    
    final_result_dict['type_of_data'].append(synthetic_filename_info['type_of_data'])
    final_result_dict['dataset'].append(synthetic_filename_info['dataset_name'])
    final_result_dict['drift'].append(synthetic_filename_info['drift_type'])
    final_result_dict['width'].append(synthetic_filename_info['drift_width'])
    final_result_dict['encoding'].append(encoding)
    final_result_dict['n_init'].append(100)
    final_result_dict['max_iter'].append(850)
    final_result_dict['tol'].append(0)
    final_result_dict['FPR_mean'].append(fpr_mean)
    final_result_dict['latency_mean'].append(latency_mean)
    
final_result_df = pd.DataFrame.from_dict(final_result_dict)
sorted_final_result_df = final_result_df.sort_values(['drift', 'dataset', 'encoding', 'width'])
final_result_df.to_csv('sea_jupyter_results2.csv', index=False)